# 00. 🔍 Intenção de Pesquisa

asd

# 01. 📑 Importações

In [52]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import faiss
import numpy as np

import warnings
warnings.filterwarnings("ignore")

# 02. 📖 Lendo Dados

In [37]:
df = pd.read_csv('../data/01_raw/raw.csv', index_col='Unnamed: 0')
df.head()

,statement,status
0,oh my gosh,Anxiety
1,"trouble sleeping, confused mind, restless hear...",Anxiety
2,"All wrong, back off dear, forward doubt. Stay ...",Anxiety
3,I've shifted my focus to something else but I'...,Anxiety
4,"I'm restless and restless, it's been a month n...",Anxiety


In [38]:
df = df.dropna(subset=['statement', 'status'])
df.info()

<class 'pandas.DataFrame'>
Index: 52681 entries, 0 to 53042
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   statement  52681 non-null  str  
 1   status     52681 non-null  str  
dtypes: str(2)
memory usage: 1.2 MB


In [39]:
# df = df.sample(1000, random_state=777)

# 03. 🧮 Dividindo em Treino e Teste e Analisando

In [40]:
X_train, X_test, y_train, y_test = train_test_split(df['statement'], df['status'], test_size=0.2, random_state=777, stratify=df['status'])

In [41]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((42144,), (10537,), (42144,), (10537,))

In [42]:
y_train.value_counts()

status
Normal                  13074
Depression              12323
Suicidal                 8521
Anxiety                  3073
Bipolar                  2221
Stress                   2070
Personality disorder      862
Name: count, dtype: int64

# 04. Pré-processando

In [45]:
model = SentenceTransformer('all-MiniLM-L6-v2')

Loading weights: 100%|██████████| 103/103 [00:00<00:00, 2729.20it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [50]:
X_train_emb = model.encode(X_train.tolist())

# 05. 🧪 Modelagem

## Regressão Logística

In [51]:
param_grid = [
    {
        "solver": ["lbfgs"],
        "penalty": ["l2"],
        "C": [0.1, 1, 5, 10, 20]
    },
    {
        "solver": ["liblinear"],
        "penalty": ["l1", "l2"],
        "C": [0.1, 1, 5, 10, 20]
    },
    {
        "solver": ["saga"],
        "penalty": ["elasticnet"],
        "l1_ratio": [0.1, 0.3, 0.5, 0.7, 0.9],
        "C": [0.1, 1, 5, 7, 10, 15, 20]
    }
]

lr = LogisticRegression()

grid_search = GridSearchCV(estimator=lr, param_grid=param_grid, cv=5, n_jobs=-1, verbose=False, scoring='f1_macro')

grid_search.fit(X_train_emb, y_train)

print("Best Hyperparameters:", grid_search.best_params_)
print("Best F1 Score:", grid_search.best_score_)
print("CV Scores:", grid_search.cv_results_['mean_test_score'])

Best Hyperparameters: {'C': 10, 'l1_ratio': 0.9, 'penalty': 'elasticnet', 'solver': 'saga'}
Best F1 Score: 0.7160842451422631
CV Scores: [0.63625253 0.6997966  0.70902847 0.70812344 0.70781167        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan 0.63130846 0.62118212 0.60895465
 0.59775695 0.59003728 0.70102463 0.70218225 0.70384706 0.70451704
 0.7036197  0.71304934 0.7138509  0.7138115  0.71482429 0.71412674
 0.714654   0.71481308 0.71464937 0.7146421  0.71435469 0.71513762
 0.71528141 0.71489339 0.71500679 0.71608425 0.71484406 0.71530207
 0.71576303 0.71602112 0.71534709 0.71552526 0.71601532 0.71592376
 0.71559345 0.71484502]


## K-NN no vector DB

## RAG no vector DB